# Deep riccy project

## Env creation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

! tar -zxf /content/drive/MyDrive/Uni/DeepRiccy/refcocog.tar.gz
! pip3 install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  --quiet
! pip install ftfy regex tqdm --quiet
! pip install git+https://github.com/openai/CLIP.git --quiet
! pip install rouge-metric --quiet
! pip install torchmetrics --quiet
! pip install torchvision --quiet

local_path = '/content/refcocog/images/'
local_annotations = '/content/refcocog/annotations/'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 14.5 MB/s eta 0:00:00


In [2]:
#imports

import json
import pickle
import torch
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from pkg_resources import packaging
import clip
import numpy as np

import os
import skimage
import IPython.display

from collections import OrderedDict
import torch
import torchmetrics as tm
import torchvision
from torchvision import ops 

## Baseline

### Utilities

In [3]:
def padd_image(img):
  avg_color_per_row = np.average(img, axis=0)
  avg_color = np.average(avg_color_per_row, axis=0)
  old_image_width,old_image_height  = img.size
  # create new image of desired size and color (blue) for padding
  if(old_image_height>224 or old_image_width>224):
    if(old_image_height>old_image_width):
          new_image_width = old_image_height
          new_image_height = old_image_height


    else:
          new_image_width = old_image_width
          new_image_height = old_image_width

    
    color=avg_color
    #color = (255,0,255)

    result = np.full((new_image_height,new_image_width, 3), color, dtype=np.uint8)

    # compute center offset
    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # copy img image into center of result image
    result[y_center:y_center+old_image_height, 
          x_center:x_center+old_image_width] = img   

        
  else:
    new_image_width = 224
    new_image_height = 224
    '''if(old_image_height>old_image_width):
          new_image_width = old_image_height
          new_image_height = old_image_height


    else:
          new_image_width = old_image_width
          new_image_height = old_image_width'''
 
    #color = (255,0,255)
    color=avg_color
    result = np.full((new_image_height,new_image_width, 3), color, dtype=np.uint8)

    # compute center offset
    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # copy img image into center of result image
    result[y_center:y_center+old_image_height, 
          x_center:x_center+old_image_width] = img


  img= Image.fromarray(result)
  img=img.resize((224,224))
  return img

In [4]:
def split_string(string):
    string = string.split("_")
    string = string[:-1]
    string = "_".join(string)
    append = ".jpg"
    string = string + append
    
    return string

In [5]:
def convert_bbox(bbox, img):
  x1, y1, width, height = bbox
  x2, y2 = x1 + width, y1 + height

  # Verify coordinates
  if x1 < 0 or y1 < 0 or x2 > img.width or y2 > img.height:
      print("Bounding box fuori dai limiti dell'immagine!")
  else:
    return x1, y1, x2, y2

In [6]:
def convert_bbox_no_img(bbox):
  x1, y1, width, height = bbox
  x2, y2 = x1 + width, y1 + height

  return x1, y1, x2, y2

In [7]:
def convert_yolo_bbox(bbox):
  return bbox[:4]

### Metrics

In [8]:
class Recall:
    def __init__(self):
        self.true_positives = 0
        self.false_negatives = 0

    def update(self, correct):
        if correct:
            self.true_positives += 1
        else:
            self.false_negatives += 1
    
    def compute(self):
        return self.true_positives / (self.true_positives + self.false_negatives)
    
    def reset(self):
        self.true_positives = 0
        self.false_negatives = 0

In [108]:
class Metrics:
    def __init__(self, model, trasform, treshold):
        self.transform = trasform
        #initialize torch tensor
        self.iou = torch.tensor([]).cuda()
        self.recall = Recall()
        self.model = model
        self.treshold = treshold
        self.cosine_similarity = torch.tensor([]).cuda()
        self.euclidean_distance = torch.tensor([]).cuda()

    def update(self, predicted_bbox, target_bbox, predicted_image, target_image):
        predicted_bbox = torch.tensor(predicted_bbox)
        target_bbox = torch.tensor(target_bbox)
        target_bbox = convert_bbox_no_img(target_bbox)

        with torch.no_grad():
            # Preprocess the predicted image and compute the predicted embedding
            predicted_image = padd_image(predicted_image)
            image_tensor = self.transform(predicted_image)
            image_tensor = image_tensor.unsqueeze(0).cuda()  # Add batch dimension
            predicted_embedding = self.model.encode_image(image_tensor)

            # Preprocess the target image and compute the target embedding
            target_image = padd_image(target_image)
            target_image_tensor = self.transform(target_image)
            target_image_tensor = target_image_tensor.unsqueeze(0).cuda()  # Add batch dimension
            target_embedding = self.model.encode_image(target_image_tensor)

        similarity = torch.nn.functional.cosine_similarity(predicted_embedding, target_embedding)
        distance = torch.nn.functional.pairwise_distance(predicted_embedding, target_embedding)
        
        #convert bboxes into torch tensors
        predicted_bbox = torch.tensor(predicted_bbox)
        target_bbox = torch.tensor(target_bbox)
        predicted_bbox = convert_yolo_bbox(predicted_bbox)
        actual_iou = ops.box_iou(predicted_bbox.unsqueeze(0).cuda(), target_bbox.unsqueeze(0).cuda())
        self.iou = torch.cat((self.iou, actual_iou), 0)

        #get iou value of the predicted bbox and the target bbox
        if actual_iou > self.treshold:
          self.recall.update(False)
        else:
          self.recall.update(True)
        self.cosine_similarity = torch.cat((self.cosine_similarity, similarity), 0)
        self.euclidean_distance = torch.cat((self.euclidean_distance, distance), 0)

    def to_string(self):
        mean_iou = torch.mean(self.iou)
        recall_at_05_iou = self.recall.compute()
        mean_cosine_similarity = torch.mean(self.cosine_similarity)
        mean_euclidean_distance = torch.mean(self.euclidean_distance)

        return f"Mean IoU: {mean_iou:.4f}, Recall@0.5 IoU: {recall_at_05_iou:.4f}, Mean Cosine Similarity: {mean_cosine_similarity:.4f}, Mean Euclidean Distance: {mean_euclidean_distance:.4f}"

    def reset(self):
        self.iou = torch.tensor([]).cuda()
        self.recall.reset()
        self.cosine_similarity = torch.tensor([]).cuda()
        self.euclidean_distance= torch.tensor([]).cuda()


### Dataset

In [10]:
class Coco(Dataset):
    def __init__(self, path_json, path_pickle, train=True):
        self.path_json = path_json
        self.path_pickle = path_pickle
        self.train = train

        #load images and annotations
        with open(self.path_json) as json_data:
            data = json.load(json_data)
            self.ann_frame = pd.DataFrame(data['annotations'])
            self.ann_frame = self.ann_frame.reset_index(drop=False)


        #load annotations
        with open(self.path_pickle, 'rb') as pickle_data:
            data = pickle.load(pickle_data)
            self.refs_frame = pd.DataFrame(data)

        #separate each sentence in dataframe
        self.refs_frame = self.refs_frame.explode('sentences')

        self.refs_frame = self.refs_frame.reset_index(drop=False)

        self.size = self.refs_frame.shape[0]

        self.dataset = pd.merge(self.refs_frame, self.ann_frame, left_on='ann_id', right_on='id')
        self.dataset = self.dataset.drop(columns=['segmentation', 'id', 'category_id_y','ref_id', 'index_x', 'iscrowd', 'image_id_y', 'image_id_x', 'category_id_x', 'ann_id', 'sent_ids', 'index_y', 'area'])

    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        return self.dataset.iloc[idx]

    def get_annotation(self, idx):
      return self.ann_frame.iloc[idx]
    
    def get_imgframe(self, idx):
      return self.img_frame.iloc[idx]

    def get_validation(self):
        #return the dataframe that has as attribute the validation
        return self.dataset[self.dataset['split'] == 'val']
    
    def get_test(self):
        #return the dataframe that has as attribute the test
        return self.dataset[self.dataset['split'] == 'test']
    
    def get_train(self):
        #return the dataframe that has as attribute the train
        return self.dataset[self.dataset['split'] == 'train']


In [11]:
#test dataset

dataset = Coco(local_annotations + 'instances.json', local_annotations + "refs(umd).p")

print(len(dataset))

95010


In [12]:
dataset.dataset

,split,sentences,file_name,bbox
0,test,"{'tokens': ['the', 'man', 'in', 'yellow', 'coa...",COCO_train2014_000000380440_491042.jpg,"[374.31, 65.06, 136.04, 201.94]"
1,test,"{'tokens': ['skiier', 'in', 'red', 'pants'], '...",COCO_train2014_000000380440_491042.jpg,"[374.31, 65.06, 136.04, 201.94]"
2,test,"{'tokens': ['there', 'is', 'red', 'colored', '...",COCO_train2014_000000419645_398406.jpg,"[93.95, 83.29, 504.61, 290.57]"
3,test,"{'tokens': ['a', 'shiny', 'red', 'vintage', 'p...",COCO_train2014_000000419645_398406.jpg,"[93.95, 83.29, 504.61, 290.57]"
4,test,"{'tokens': ['a', 'apple', 'desktop', 'computer...",COCO_train2014_000000478885_124383.jpg,"[338.8, 82.19, 147.34, 157.37]"
...,...,...,...,...
95005,train,"{'tokens': ['the', 'larger', 'banana', 'is', '...",COCO_train2014_000000003518_1042682.jpg,"[141.7, 20.68, 359.02, 144.57]"
95006,train,"{'tokens': ['a', 'guy', 'in', 'black', 'jacket...",COCO_train2014_000000302199_473946.jpg,"[141.34, 86.71, 115.33, 288.75]"
95007,train,"{'tokens': ['the', 'man', 'in', 'the', 'dark',...",COCO_train2014_000000302199_473946.jpg,"[141.34, 86.71, 115.33, 288.75]"
95008,train,"{'tokens': ['a', 'person', 'in', 'red', 'dress...",COCO_train2014_000000573297_472971.jpg,"[31.71, 67.03, 352.29, 572.97]"


### Model

In [58]:
class VisualGrounding(torch.nn.Module):
    def __init__(self, yolo_version, clip_version, local_path, img_path):
        super(VisualGrounding, self).__init__()
        self.local_path = local_path
        self.img_path = img_path
        #initialize models
        self.yolo = torch.hub.load('ultralytics/yolov5', yolo_version , pretrained=True)
        self.clip, self.preprocess = clip.load(clip_version)
        
        #define metrics
        treshold = 0.5
        transform = torchvision.transforms.Compose([
          torchvision.transforms.ToTensor(),
          torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])
          ])
        self.metrics = Metrics(self.clip, transform, treshold)
        
    
    def forward(self, img_path, sentence):
      max_similarity = 0
      max_image = None
      max_bbox = None

      yolo_output = self.yolo(self.local_path+img_path)

      original_img = Image.open(self.local_path+img_path).convert("RGB")

      for i in range(len(yolo_output.xyxy[0])):
          x1 = yolo_output.xyxy[0][i][0].cpu().numpy()
          x1 = np.rint(x1)
          y1 = yolo_output.xyxy[0][i][1].cpu().numpy()
          y1 = np.rint(y1)
          x2 = yolo_output.xyxy[0][i][2].cpu().numpy()
          x2 = np.rint(x2)
          y2 = yolo_output.xyxy[0][i][3].cpu().numpy()
          y2 = np.rint(y2)

          img_cropped = original_img.crop((x1, y1, x2, y2))

          img = self.preprocess(img_cropped).cuda().unsqueeze(0)
          text = clip.tokenize([sentence]).cuda()

          with torch.no_grad():
              image_features = self.clip.encode_image(img).float()
              text_features = self.clip.encode_text(text).float()

          image_features /= image_features.norm(dim=-1, keepdim=True)
          text_features /= text_features.norm(dim=-1, keepdim=True)
          similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

          if similarity > max_similarity:
              max_similarity = similarity
              max_image = img_cropped
              max_bbox = yolo_output.xyxy[0][i]

          return max_bbox, max_image

        
    def evaluate(self, img_path, sentence, gt, original_img):
      bbox = convert_bbox(gt, original_img)
      gt_crop = original_img.crop(bbox)
      prediction_bbox, prediction_img = self.forward(img_path, sentence)
      self.metrics.update(prediction_bbox, bbox, prediction_img, gt_crop)
      return prediction_bbox, prediction_img
    
    def reset_metrics(self):
      self.metrics.reset()

    def get_metrics(self):
      return self.metrics.to_string()
          
        

### Model evaluation

In [14]:
dataset = Coco(local_annotations + 'instances.json', local_annotations + "refs(umd).p")

In [109]:
model = VisualGrounding('yolov5x','ViT-B/32', local_path, local_annotations)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-9 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [110]:
#evaluation of the model
model.reset_metrics()

for i in range(0, 1000):
    input = dataset[i]
    image_path = split_string(input["file_name"])
    sentence = input["sentences"]["raw"]
    gt = input["bbox"]
    original_img = Image.open(local_path + image_path).convert("RGB")
    # print img dimensions and box coordinates
    bbox, img = model.evaluate(image_path, sentence, gt, original_img)

    if i % 100 == 0:
        print("Iteration: ", i)
        print(model.get_metrics())
        

Iteration:  0
Mean IoU: 0.0000, Recall@0.5 IoU: 1.0000, Mean Cosine Similarity: 0.8789, Mean Euclidean Distance: 5.0664
Iteration:  100
Mean IoU: 0.1636, Recall@0.5 IoU: 0.8812, Mean Cosine Similarity: 0.8378, Mean Euclidean Distance: 5.6156
Iteration:  200
Mean IoU: 0.1388, Recall@0.5 IoU: 0.9005, Mean Cosine Similarity: 0.8160, Mean Euclidean Distance: 6.0979
Iteration:  300
Mean IoU: 0.1567, Recall@0.5 IoU: 0.9070, Mean Cosine Similarity: 0.8210, Mean Euclidean Distance: 5.9462
Iteration:  400
Mean IoU: 0.1545, Recall@0.5 IoU: 0.9202, Mean Cosine Similarity: 0.8176, Mean Euclidean Distance: 5.9997
Iteration:  500
Mean IoU: 0.1716, Recall@0.5 IoU: 0.9062, Mean Cosine Similarity: 0.8211, Mean Euclidean Distance: 5.8832
Iteration:  600
Mean IoU: 0.1806, Recall@0.5 IoU: 0.8985, Mean Cosine Similarity: 0.8260, Mean Euclidean Distance: 5.7603
Iteration:  700
Mean IoU: 0.1783, Recall@0.5 IoU: 0.8959, Mean Cosine Similarity: 0.8245, Mean Euclidean Distance: 5.7852
Iteration:  800
Mean IoU: 